# Import Library

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow_metadata.proto.v0 import schema_pb2
import pprint
from tensorflow_transform.tf_metadata import dataset_metadata
import tensorflow_transform as tft
from tensorflow_transform import TFTransformOutput

Insight: 
* Memanggil library yang dibutuhkan dalam project

# Load model dari folder output trainer

In [3]:
model_path = "pipelines/insurance_pipeline/Trainer/model/15/serving_model_dir"
loaded_model = tf.keras.models.load_model(model_path)
tfrecord_path = "pipelines/insurance_pipeline/Transform/transformed_examples/5/train/transformed_examples-00000-of-00001.gz"

2025-07-09 12:56:10.484849: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2
2025-07-09 12:56:10.540728: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2592055000 Hz
2025-07-09 12:56:10.546640: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555556901780 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-07-09 12:56:10.546710: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Insight: 
* Memanggil model yang sudah di export sebelumnya
* Serta melakukan path ke file TFRecord hasil Transform

# Prediksi Model

In [12]:
raw_dataset = tf.data.TFRecordDataset(
    tf.io.gfile.glob('pipelines/insurance_pipeline/Transform/transformed_examples/5/train/*.gz'),
    compression_type='GZIP'
)

tft_output = tft.TFTransformOutput('pipelines/insurance_pipeline/Transform/transform_graph/5')
feature_spec = tft_output.transformed_feature_spec()

def parse_example(record):
    return tf.io.parse_single_example(record, feature_spec)

parsed = raw_dataset.map(parse_example) 
parsed = parsed.batch(1)

Insight:
* Membaca dan mem-parse data hasil transformasi dari file TFRecord yang telah di-scale dan di-encode.
* Output-nya digunakan sebagai input yang sudah siap untuk dimasukkan ke model prediksi.

In [13]:
def _dense_cast(x, y):
    result = {}
    for key, value in x.items():
        if isinstance(value, tf.SparseTensor):
            value = tf.sparse.to_dense(value)
        result[key] = tf.cast(value, tf.float32)

    if isinstance(y, tf.SparseTensor):
        y = tf.cast(tf.sparse.to_dense(y), tf.float32)
    else:
        y = tf.cast(y, tf.float32)

    return result, y

Insight:
* Fungsi _dense_cast digunakan untuk mengubah input bertipe SparseTensor menjadi DenseTensor dan bertipe float32.
* Hal ini penting agar data bisa diproses dengan baik oleh model Keras saat prediksi.

In [17]:
parsed_dataset = parsed.map(lambda x, y: _dense_cast(x, y))
for x, y in parsed_dataset.take(1):
    input_dict = {k: tf.expand_dims(v, axis=0) for k, v in x.items()}
    prediction = loaded_model.predict(input_dict)
    print("Predicted charges:", prediction)

Predicted charges: [[0.2179867]]


Insight:
* Proses ini melakukan prediksi dengan model terhadap data yang telah ditransformasi.
* Data diproses menjadi dictionary (input_dict) yang sesuai format input model, lalu diprediksi dan hasilnya ditampilkan.